# 1. 모델과 토크나이저 불러오기 

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy

# 사용하는 연산장치 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print("This notebook is running on", device)

# huggingface를 통해 모델과 토크나이저 불러오기
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

This notebook is running on cuda


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 토크나이저 확인하기 

In [2]:
# 모델 파라미터 크기별 사용하는 토큰 수 확인하기
tokenizer.max_model_input_sizes

{'gpt2': 1024,
 'gpt2-medium': 1024,
 'gpt2-large': 1024,
 'gpt2-xl': 1024,
 'distilgpt2': 1024}

In [3]:
# 예제 문장
input_txt = "바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."

In [4]:
# 문장 토큰화
tokens = tokenizer(input_txt).tokens()

# 토큰 정수화
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].numpy()

In [5]:
# 토큰과 해당하는 정수값 매칭을 DataFrame으로 보여주기
pd.options.display.max_columns = 40
pd.options.display.max_rows = 60
df = pd.DataFrame([tokens, input_ids[0]], index=["kogpt-2_tokens", "Input_IDs"])
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
kogpt-2_tokens,▁바람,도,▁없는,▁공중에,▁수직,의,▁파,문을,▁내,이며,▁고,요,히,▁떨어지는,▁오동,잎은,▁누,구의,▁발자,취,▁입,니까,.
Input_IDs,10891,7235,9712,49207,14438,8143,9203,9941,9094,9639,9065,8084,8811,21215,34769,19985,9669,10139,21626,8408,9241,23775,389


## 모델 디코딩 확인하기

In [6]:
# 예제 문장 토큰화
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)

# Beam Search Decoding
max_length = 128
output_beam = model.generate(input_ids, # 입력 시퀀스
                             max_length=max_length, # 생성 시퀀스 최대 길이
                             num_beams=7, # Beam Search할 범위
                             no_repeat_ngram_size=2, # 지정된 ngram 단위로 중복 체크
                             do_sample=True, # 토큰 샘플링
                             temperature=2.0, # 토큰 결정시 확률 반영도
                             top_k=50, # 후보 토큰 고를시 높은 확률순 k위까지만 보고 결정
                            )

# beam search로 만들어진 문장중 하나 출력
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까." "도대체 누가 너의 발목을 잡아넣었는지 샅샅이 수색해서 밝혀내야 할 것이네."
"정확한 것은 네게 알려주마."
"허허, 그건. 어서 빨리 말해봐야지. 왜 그러냐고 묻기라도 하면 자네만 가만 안 둘 거라구. 알았어. 네가 내 말을 듣고 무슨 짓을 한 것인지는 모르지만, 내가 말하기 전에 말이야. "그럼, 이게 무슨 소리야?"



# 2. 데이터셋 불러오기

## SFT(Supervised Fine-Tuning) 데이터셋 
- prompt
    - 사용자가 올릴 질문
- completion
    - 모델이 생성해야할 답변
- tokens
    - 총 토큰 수로 추정
    - KoChatGPT가 학습할 때 사용할 tokenizer 기준으로 보임

In [7]:
import os
import json 

HOME_DIR = os.getenv('HOME')

data_path_1_SFT = HOME_DIR + '/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl' 
with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

# SFT 데이터셋 크기
print('SFT 데이터셋 크기:', len(list_data_dict))

# SFT 데이터셋 확인하기
list_data_dict[:3]

SFT 데이터셋 크기: 12000


[{'prompt': '불고기용 고기 한우에요?',
  'completion': "'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.",
  'tokens': 193},
 {'prompt': '쓰던 앱이 유료로 전환됐어',
  'completion': "'어떤 앱인지 모르기 때문에 정확한 답변을 드리기 어렵습니다. 하지만, 일반적으로 유료 전환된 앱은 기존 무료 앱에서 추가적인 기능이나 서비스를 제공하는 경우가 많습니다. 이 경우, 유료 전환 이전 가격이 매우 저렴하거나 무료인 경우가 많으므로, 앱을 계속 사용하려면 추가적인 비용을 지불해야 할 수 있습니다. 하지만, 유료 전환 이후에는 기존 사용자 또는 새로운 사용자 모두가 추가 기능과 높은 품질의 서비스를 이용할 수 있게 됩니다. 따라서, 앱 개발자는 유료 모델로 전환함으로써 앱의 수익을 증가시키고 더 나은 서비스를 제공할 수 있습니다.",
  'tokens': 288},
 {'prompt': '여친이랑 다툼',
  'completion': "'저는 인공지능이기 때문에 여친과의 다툼에 대해서는 조언을 드릴 수는 없습니다. 하지만 다투게 되면 상대방의 말을 경청하는 태도와 서로의 감정을 존중하는 것이 중요합니다. 논리적이고 차분한 대화를 통해 문제를 해결해보세요. 그리고 서로를 이해하고 서로의 의견을 수용하는 것이 중요합니다.",
  'tokens': 153}]

## RM (Reward Model) 데이터셋
- prompt
    - 사용자가 올릴 질문
- completion_0, completion_1, completion_2
    - 각 모델별 답변
        - ChatGPT
        - GPT3(Ada)
        - GPT3(Davinci)
    - 어떤 답변이 어떤 모델로 생성됐는지 알수 없도록 답변 순서를 섞음
- ranking
    - 사람이 라벨링한 각 답변의 품질 랭킹
    - 값이 낮을수록 높은 품질

In [8]:
data_path_2_RM = HOME_DIR + '/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl'
with open(data_path_2_RM, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

# RM 데이터셋 크기
print("RM 데이터셋 크기:", len(list_data_dict))

# RM 데이터셋 확인하기
list_data_dict[:3]

RM 데이터셋 크기: 10220


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?',
  'completion_0': 'Allow me to answer your question. I know that you are curious about me.',
  'completion_1': '번디는 다양한 인터뷰자들과 뉴스홍보 담당자들과의 면담 때 밝혔다.',
  'completion_2': '라이언에게 말했다.',
  'ranking': [2, 1, 0]},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?',
  'completion_0': '개포주공아파트는 다섯 단지로 이루어져 있습니다.',
  'completion_1': '이날 목송에서 구글상위노',
  'completion_2': '개포주공아파트는 총 27개 단지로 이루어져 있습니다.',
  'ranking': [2, 0, 1]},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?',
  'completion_0': 'The diameter of the Metallic domain is bigger than the Hyperonic domain.',
  'completion_1': '이 질문은 조금 불분명합니다. 김영삼 대통령이 후보 시절에 어떤 발언을 했고, 누가 그 발언을 문제삼았는지에 따라 답이 다를 수 있습니다.\\n\\n만약 김영삼 대통령이 후보 시절에 지역표심을 겨냥한 발언을 했다는 가정하에, 그 발언을 문제삼은 후보가 누구였는지를 대답하자면, 그 답은 이화선 당시 민주당 대통령 후보가 될 것입니다. 1992년 총선 때, 김영삼 대선후보는 "집값이 오른 노량진역 부근의 부동산 가격은 세월호 폭침 후 \\\'강남 도시재생\\\' 일환으로 상승했다"는 발언을 했습니다. 하지만 이화선 후보는 이 발언을 "전국적으로 경제적 발전이 이루어지지 않은 지방민의 마음을 멀리해지려는 무례한 발언"이라고 비판하며 문

## PPO 데이터셋
- prompt
    - 사용자가 전달할 질문
    - SFT한 모델이 답변을 생성하고 RM이 주는 reward에 따라 학습하기 때문에 prompt밖에 없음

In [9]:
data_path_3_PPO = HOME_DIR + '/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl'
with open(data_path_3_PPO, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

# PPO 데이터셋 크기
print("PPO 데이터셋 크기:", len(list_data_dict))

# PPO 데이터셋 확인하기
list_data_dict[:3]

PPO 데이터셋 크기: 12000


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?'},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?'},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?'}]

# 3. Supervised Fine-Tuning
- Foundation 모델이 일단 원하는 Downstream Task를 해결할 수 있도록 Fine-Tuning
    - ![SFT](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/rlhf/pretraining.png)

In [10]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
import json
from dataclasses import dataclass

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## STF를 수행할 모델과 토크나이저 불러오기

In [11]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')

tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', # 토크나이저를 가져올 모델
                                          bos_token='</s>',     # 시작 토큰
                                          eos_token='</s>',     # 종료 토큰
                                          unk_token='</s>',     # OOV 토큰
                                          pad_token='</s>',     # 패딩 토큰
                                          padding_side="right", # 패딩 방향
                                          model_max_length=512, # 입력 문장 최대 길이
                                         )

print(tokenizer)

GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=True)


## STF 데이터셋 클래스 정의하기

In [12]:
from typing import Optional, Dict, Sequence


class SFT_dataset(Dataset): # Pytorch Dataset 클래스
    def __init__(self, 
                 data_path_1_SFT: str, # SFT 데이터가 들어있는 파일 위치
                 tokenizer: transformers.PreTrainedTokenizer, # 토크나이저
                 verbose=False,
                ):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        # 질문 문장과 대답 문장 column 이름
        pattern_instruction = 'prompt' # 원본 질문 문장
        pattern_output = 'completion' # 원본 대답 문장

        # json 데이터셋 파일 불러오기
        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        # 지시문 정의
        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }
        prompt_input = PROMPT_DICT["prompt_input"]

        # 질문 문장 전처리
        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example) # 지시문 추가
            sources.append(tmp)

        # 대답 문장 전처리
        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}") # 종료 토큰 추가
        examples = [s + t for s, t in zip(sources, targets)] # 질문 문장과 대답 문장 연결

        # 문장 토큰화
        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # 질문
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # 질문 + 대답

        # 시퀀스 정수화
        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100 # 질문 + 대답 시퀀스에서 질문 부분에 마스킹
                                      # Pytorch에서 마스크 값이 -100로 지정되어 있음

        # 데이터셋 생성
        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, 
                     strings: Sequence[str], # 토큰화할 문장 리스트
                     tokenizer: transformers.PreTrainedTokenizer, # 토크나이저
                    ) -> Dict:
        """
        문장을 토큰화해 시퀀스 정보를 딕셔너리에 담아 변환
        """
        # 문장 토큰화
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        
        # 정수 시퀀스
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        
        # 시퀀스 길이
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() # pad_token이 아닌 토큰 개수
            for tokenized in tokenized_list
        ]
        
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        """
        데이터셋 크기 반환
        """
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        """
        데이터셋의 샘플 반환
        """
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [13]:
# GPT2 모델이 학습할 수 있도록 SFT 데이터셋에 attention mask 추가
@dataclass
class DataCollatorForSupervisedDataset(object): 
    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        
        # input_ids와 labels 패딩 추가
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id), # 어텐션 마스크, 패딩 토큰 제외
        )

In [14]:
SFT_DATASET_PATH = HOME_DIR + '/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl'

train_dataset = SFT_dataset(data_path_1_SFT=SFT_DATASET_PATH, tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

# 최종 데이터셋의 샘플 확인
print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

input : tensor([  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
         8006, 14914,   375,  7673, 20479,  8091, 22311,  9036, 30902, 13675,
          375,   378,   378,   378, 41951,   454,  9549, 20549,   383,  8142,
         7192, 14914,   382, 37767, 13753,  8263,  7166,   739,  8352,  7659,
         9594, 25585, 13600,  8022,  9378, 11532,  9887, 11218,  9111, 16691,
        10351, 10561,  9128, 20479,  8091,  9065,  9446,  9036, 28420, 26521,
        10163, 26367,  6958,  9030,  9882, 12317, 25882,  9209, 37194, 10351,
         9036, 12168, 10529, 15989,  9719, 15434, 10552, 11188, 13362,  9036,
        15805, 11300, 11846,  9146, 16691,  9181,  7397, 15806, 13480, 11342,
        17596,  9161, 19996,  9025, 25006, 18595,  9966, 12592, 10751, 11814,
         8711,  9046, 12450,  9117,  7377, 12521,     1])
output: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -10

In [15]:
# 디코딩 후 확인
print('input_ids:')
print(tokenizer.decode(train_dataset.input_ids[0]))
print('\n\n')

print('labels:')
for token in train_dataset.labels[0]:
    if token != -100:
        print(tokenizer.decode(token), end=' ')
print()

input_ids:
### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.</s>



labels:
' 저는 인공 지 능  챗 봇 이며, 직접적으로 식품 에 관한 정보를 가지고 있지 않 습니다. 하지만 일반적으로 불 고기 용 고 기는 한 우, 쇠고 기, 돼지고 기 등 다양한 종류의 고기를 사용 합니다. 하지만 한 우는 대표적인 고급 육 류로 알려져 있기 때문에, 한 우를 사용하는 경우도 많 습니다. 알 러 지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택 해 주 시기 바 랍 니다. </s> 


## SFT Trainer 정의

In [16]:
training_args = TrainingArguments(
    output_dir="aiffel/KoChatGPT/test", # checkpoint를 저장할 위치
    overwrite_output_dir=True, # 덮어쓰기 허용
    num_train_epochs=1, # 학습 epoch 수
    per_device_train_batch_size=8, # 연산 장치당 학습 배치 크기
    per_device_eval_batch_size=8, # 연산 장치당 평가 배치 크기
    warmup_steps=5, # warmup steps, 학습 시작시 0부터 learning_rate까지 천천히 높이는 step 수
    prediction_loss_only=True, # loss값만 반환
    fp16 = True # 가중치는 fp16 타입 사용
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

## SFT 학습

In [17]:
trainer.train()
model.save_pretrained('aiffel/KoChatGPT/output_1_SFT')

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,2.984100
1000,2.776800
1500,2.687200


## SFT 결과 확인 

In [18]:
# 모델을 통해 문장 생성하는 pipeline
generator = pipeline('text-generation', model='aiffel/KoChatGPT/output_1_SFT', tokenizer=tokenizer)

# generator.__call__()에 전달할 파라미터
generation_args = dict(   
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n   
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

# 지시문
PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

# 확인용 질문 문장
list_prompt = ['불고기용 고기 한우에요?',
               '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
               '시카고 오헤어 국제공항은 어디에 있어?',
               '오늘 미세먼지 어때?']

# 질문 문장에 지시문 추가
list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

# SFT을 진행한 모델 추론
list_result = generator(list_prompt, **generation_args)   
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))

/opt/conda/lib/python3.9/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(



### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에 불고기용 고기의 종류와 양에 대한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기는 쇠고기와 함께 먹는 음식 중 하나입니다. 따라서 불고기를 먹을 수 있는 종류는 다양합니다. 예를 들어, 닭가슴살 스테이크, 오므라이스 샐러드 등이 있습니다.

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'리처드 닉슨은 42대 부통령직을 수행했습니다.作)作)은 "리처드 닉슨"이 41대 부통령을 수행한 년도를 가리키는 말입니다.作)는 "리처드 닉슨"이 40대 부통령을 맡았던 년도를 의미합니다.作은 "리처드슨"이 50대 부통령

### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어?

### Response(응답):'시카고 오 헤어 국제공항은 미국 캘리포니아주 샌프란시스코에 위치해 있습니다.子供共和國際空港)이라고 불립니다.子供公和国際空港이라는 뜻입니다.子供空和國際公港이라는 이름을 가진 항공사는 다음과 같습니다.\n\n1. 대한항공

### Instruction(명령어):
오늘 미세먼지 어때?

### Response(응답):'저는 인공지능 챗봇으로써 미세먼지 정보를 알 수 없습니다. 미세먼지 예보를 확인해 보시는 것이 좋겠습니다.\n\n미세먼지 예보: 일반적으로 미세먼지는 주로 중국에서 발원하여 중국 전역으로 퍼져나가기 때문에 중국발 미세먼지가 유입될


In [19]:
# 다음 모델 학습을 위해 할당한 GPU 메모리 초기화
torch.cuda.empty_cache()

# 4. Reward Model
- 어떤 답변이 더욱 선호되는지 평가하는 모델 만들고 학습시키기
    - ![RM](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/rlhf/reward-model.png)

In [20]:
!pip install "$HOME/aiffel/KoChatGPT/colossalai_ChatGPT_230319/"

Processing /aiffel/aiffel/KoChatGPT/colossalai_ChatGPT_230319
  Preparing metadata (setup.py) ... done


  Created wheel for chatgpt: filename=chatgpt-0.1.0-py3-none-any.whl size=46664 sha256=e01a4b16d46a3b25be1dca74747760c4fa0430beaea224f672953f80884b1548
  Stored in directory: /tmp/pip-ephem-wheel-cache-ohr49ub2/wheels/79/25/c3/338e0c56a2253a8ea6c41e8692f6eb2409a3898c63b234b103
Successfully built chatgpt
  Attempting uninstall: chatgpt
    Found existing installation: chatgpt 0.1.0
    Uninstalling chatgpt-0.1.0:
      Successfully uninstalled chatgpt-0.1.0


In [21]:
import os
import json
from typing import Optional
import torch
import torch.nn as nn
from torch.optim import Adam
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.trainer import RewardModelTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, AutoConfig
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model
import loralib as lora

## RM 모델 클래스 정의

In [22]:
class GPTRM_custom(RewardModel):
    def __init__(self,
                 pretrained: Optional[str] = None, # pre-trained 모델
                 config: Optional[GPT2Config] = None, # pre-trained 모델 설정
                 checkpoint: bool = False, # 체크포인트 저장
                 lora_rank: int = 0, # 학습에 lora 적용시 rank 수
                 lora_train_bias: str = 'none', # lora 및 pre-trained 모델의 bias 학습 적용 여부
                 tokenizer=None # 토크나이저
                ) -> None:
        # 모델 불러오기
        if pretrained is not None: 
            # 지정된 pre-trained 모델 불러오기
            model = GPT2Model.from_pretrained(pretrained)
            model.resize_token_embeddings(len(tokenizer))
        elif config is not None: 
            # 지정된 config 적용해 GPT2 모델 생성
            model = GPT2Model(config)
        else:
            # 기본 설정으로 GPT2 모델 생성
            model = GPT2Model(GPT2Config())
        if checkpoint:
            # checkpoint 불러오기
            model.gradient_checkpointing_enable()

        # reward가 스칼라값으로 나오도록 Head 설정
        value_head = nn.Linear(model.config.n_embd, 1)
        super().__init__(model, value_head, lora_rank, lora_train_bias)

        # 모델 멤버 변수에 저장
        if pretrained is not None:
            self.model = model
            self.pretrained = pretrained


    def save_pretrained(self, dir):
        """
        학습한 Reward Model 가중치 저장하기
        """
        if self.pretrained is not None:
            self.model.save_pretrained(dir)

## RM 생성

In [23]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

# RM 불러오기
with NaiveStrategy().model_init_context(): # Single GPU 환경
        model = GPTRM_custom(pretrained='skt/kogpt2-base-v2', lora_rank=0, tokenizer=tokenizer).cuda()

Some weights of the model checkpoint at skt/kogpt2-base-v2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## RM 데이테셋 생성

In [24]:
with open(HOME_DIR + '/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

total_data_ranking2chosen = []
for tmp in list_data_dict:
    one_data_ranking2chosen = []

    # 둘 중 랭킹이 높은 (값이 낮은) 답변을 chosen으로 저장
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    # 이하 동문
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    # 이하 동문
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)

    total_data_ranking2chosen.extend(one_data_ranking2chosen)

# RM 데이터셋 가공 전 크기
print('before data num: %d'%(len(list_data_dict)))
# RM 데이터셋 가공 후 크기
print('after  data num: %d'%(len(total_data_ranking2chosen)))
# RM 데이터셋 가공된 샘플
print('data example: \n%s'%total_data_ranking2chosen[45])

before data num: 10220
after  data num: 30660
data example: 
{'prompt': '애플은 리사를 어떻게 처리했어', 'chosen': '애플이 누구인지 명확히 알 수 없어서, 리사가 누구인지와 어떤 상황에서 처리되었는지에 대한 추가적인 정보가 필요합니다. 따라서, 보다 정확한 답변을 제공할 수 없습니다.', 'rejected': '애플은 리사를 위해 고객 서비스 부서에서 고객 다양한 컴퓨터 관련 문제에 대해 응답하는 데 필요한 모든 지원을 제공했습니다. 사용자가 하드웨어 문제를 경험할 때, 전문가들은 필요한 수리(수리, 추가 부품 제공, 소프트웨어 업그레이드 등)을 제공해 드릴 수 있습니다. 또한, 사용자가 사용 방법 문제나 기타 문제를 경험할 때, 대화 상대로 사용자를 지원할 수 있는 전문 고객 서비스 직원들이 사용자에게 상담하고 도움을 주는 데 도움이 될 수 있는 정보를 제공합니다. 또한, 인터넷에서 제공되는 정보를 통해 문제를 해결하거나 고객 서비스 웹 사이트를 통해 자신의 문제를 진단할 수 있도록 하는 등 다양한 방법으로 리사를 처리해 왔습니다.'}


In [25]:
import random

random.seed(230319)

# 데이터셋 순서 섞기
random.shuffle(total_data_ranking2chosen)
print(total_data_ranking2chosen[45])

{'prompt': '유아인이 류승완 감독을 만나 영화 베테랑의 시나리오를 받았던 곳은?', 'chosen': '유아인이 류승완 감독을 만나 영화 베테랑의 시나리오를 받았던 곳은 류승완의 사무실입니다.', 'rejected': '대구 영화사옥'}


In [26]:
# train/test split
train_data = total_data_ranking2chosen[:1000] 
eval_data = total_data_ranking2chosen[1000:1200]

# train/test 데이터셋 크기
print(len(train_data))
print(len(eval_data))

# RewardDataset으로 Wrapping
train_dataset = RewardDataset(train_data, tokenizer, 512)
eval_dataset = RewardDataset(eval_data, tokenizer, 512)

1000
200


100%|██████████| 200/200 [00:00<00:00, 1010.95it/s]


In [27]:
# RewardDataset 확인하기 
idx = 1
print('#'*70)
print('## prompt ##')
print(train_data[idx]['prompt'])
print('#'*70)
print('## chosen ##')
print(train_data[idx]['chosen'])
print('#'*70)
print('## rejected ##')
print(train_data[idx]['rejected'])

######################################################################
## prompt ##
흑고래의 무게는 어느 정도야
######################################################################
## chosen ##
흑고래의 평균 몸무게는 약 25~40톤 정도이지만, 최대 몸무게는 50톤 이상에 이를 수 있습니다.
######################################################################
## rejected ##
흑고래의 무게는 매우 다양하게 달라집니다. 약 200kg에서 10톤까지 달라질 수 있습니다.


## RM Trainer 정의

In [28]:
trainer = RewardModelTrainer(model=model,
                             strategy=NaiveStrategy(),
                             optim=Adam(model.parameters(), lr=5e-5),
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=4,
                             max_epochs=1)

## RM 학습

In [29]:
trainer.fit(use_lora=0)

model.save_pretrained('aiffel/KoChatGPT/output_2_RM')

Train step of epoch 0:  19%|█▉        | 47/250 [00:41<02:59,  1.13it/s, loss=0.437]


Train step of epoch 0:  38%|███▊      | 94/250 [01:22<02:16,  1.14it/s, loss=0.856]


Train step of epoch 0:  56%|█████▌    | 140/250 [02:03<01:36,  1.14it/s, loss=0.494]


Train step of epoch 0:  75%|███████▍  | 187/250 [02:44<00:55,  1.14it/s, loss=0.378]


Train step of epoch 0:  93%|█████████▎| 233/250 [03:24<00:14,  1.14it/s, loss=0.501]


Train epoch: 100%|██████████| 1/1 [03:54<00:00, 234.26s/it]


## RM 학습 결과 확인하기

In [30]:
def inference_RM(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    output = model(input_ids)
    output_reward = output.cpu().detach().numpy()[0]

    print('input: %s\nreward score: %.1f'%(input_text, output_reward))

    return output_reward

In [31]:
input_text = '인공지능은 똥멍청이 입니다'

output_reward = inference_RM(input_text=input_text)

input: 인공지능은 똥멍청이 입니다
reward score: -0.6


In [32]:
input_text = '인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다.'

output_reward = inference_RM(input_text=input_text)

input: 인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다.
reward score: -0.6


In [33]:
input_text = "인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다. AI는 현대적인 컴퓨팅 혁신에서 중추적인 역할을 하며 개인과 비즈니스의 가치를 창출합니다. 예를 들어 광학 문자 인식(OCR)은 AI를 사용해 이미지 및 문서에서 텍스트 및 데이터를 추출하고, 구조화되지 않은 콘텐츠를 비즈니스에 바로 사용할 수 있게 만들고, 유용한 정보를 창출합니다."

output_reward = inference_RM(input_text=input_text)

input: 인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다. AI는 현대적인 컴퓨팅 혁신에서 중추적인 역할을 하며 개인과 비즈니스의 가치를 창출합니다. 예를 들어 광학 문자 인식(OCR)은 AI를 사용해 이미지 및 문서에서 텍스트 및 데이터를 추출하고, 구조화되지 않은 콘텐츠를 비즈니스에 바로 사용할 수 있게 만들고, 유용한 정보를 창출합니다.
reward score: -0.4


In [34]:
input_text = "인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다. AI는 컴퓨터 공학, 데이터 분석 및 통계, 하드웨어 및 소프트웨어 엔지니어링, 언어학, 신경 과학은 물론 철학과 심리학을 포함하여 여러 학문을 포괄하는 광범위한 분야입니다. 비즈니스의 운영 수준에서 AI는 주로 머신러닝과 딥 러닝을 기반으로 하는 기술 모음으로, 데이터 분석, 예상 및 예측, 객체 분류, 자연어 처리, 추천, 지능형 데이터 가져오기 등을 수행할 수 있습니다."

output_reward = inference_RM(input_text=input_text)

input: 인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다. AI는 컴퓨터 공학, 데이터 분석 및 통계, 하드웨어 및 소프트웨어 엔지니어링, 언어학, 신경 과학은 물론 철학과 심리학을 포함하여 여러 학문을 포괄하는 광범위한 분야입니다. 비즈니스의 운영 수준에서 AI는 주로 머신러닝과 딥 러닝을 기반으로 하는 기술 모음으로, 데이터 분석, 예상 및 예측, 객체 분류, 자연어 처리, 추천, 지능형 데이터 가져오기 등을 수행할 수 있습니다.
reward score: -0.3


In [35]:
# 다음 모델 학습을 위해 할당한 GPU 메모리 초기화
torch.cuda.empty_cache()

# PPO
- 학습한 RM을 통해 SFT한 모델을 한번더 Fine-Tuning 하기
    - ![PPO](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/rlhf/rlhf.png)

In [36]:
from copy import deepcopy

import torch
from torch.optim import Adam
from chatgpt.models.base import RewardModel
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from transformers import AutoTokenizer

## 모델과 옵티마이저 준비

In [37]:
with NaiveStrategy().model_init_context(): # Single GPU 환경
    # actor, 문장을 생성할 SFT된 모델
    actor = GPTActor(pretrained=HOME_DIR + '/aiffel/KoChatGPT/output_1_SFT', lora_rank=0).to(torch.cuda.current_device())
    # critic, 생성된 문장을 reward 값으로 평가할 RM
    critic = GPTCritic(pretrained=HOME_DIR + '/aiffel/KoChatGPT/output_2_RM', lora_rank=0).to(torch.cuda.current_device())

    # 토크나이저
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", 
        model_max_length=512
    )

    # 비교군 모델
    initial_model = deepcopy(actor)
    # PPO로 학습할 모델
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

In [38]:
# actor를 학습시킬 옵티마이저
actor_optim = Adam(actor.parameters(), lr=5e-6)
# critic을 학습시킬 옵티마이저
critic_optim = Adam(critic.parameters(), lr=5e-6)

In [39]:
# Single GPU 환경에 맞춰 모델과 옵티마이저 세팅
(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

## PPO 데이터셋 준비

In [40]:
with open(HOME_DIR + '/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

In [41]:
# 토크나이징 함수 확인
print(tokenize_fn('It takes something more than intelligence to act intelligently.'))

{'input_ids': tensor([[47311, 10448, 19008,  9792, 11780, 11308, 30190, 10929, 11849, 21663,
         44389,  9574, 13799,   458, 14308, 12778, 22469, 20938, 44696,   458,
         13799,   458, 14308, 12778, 11756, 18944,   389]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')}


In [42]:
print("PPO 데이터셋 크기:", len(list_prompt))

PPO 데이터셋 크기: 12000


## PPO Trainer 정의

In [43]:
trainer = PPOTrainer(NaiveStrategy(),
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=1,  
                     train_batch_size=8, 
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

## PPO 학습 

In [44]:
trainer.fit(list_prompt, 
            num_episodes=10,  
            max_timesteps=3,
            update_timesteps=3)

model.save_pretrained('aiffel/KoChatGPT/output_3_PPO')

Episode [10/10]: 100%|██████████| 3/3 [00:17<00:00,  5.86s/it]


## PPO 학습 결과 확인하기

In [45]:
# 답변 생성 함수
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=250,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print()
    print(output)
    return output

# 지시문
PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

# 테스트용 질문 문장
list_prompt = [
    '불고기용 고기 한우에요?', 
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?', 
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?']

# 질문 문장에 지시문 추가
list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

# 질문에 따른 답변 생성
for input_text in list_prompt:
    output = generation(input_text)


### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 챗봇이므로 제가 어떤 종류의 쇠고기를 판매하는지 알 수 없습니다. 죄송합니다. srkin (쇠고기) 추가 정보가 필요합니다. 불고기용 한우는 불고기용 부위를 말합니다. geos, 또는 kinhungsin에서 각각 다른 이름입니다. geos, 또는 geos, 혹은 kinhungsin에서는 다양한 이름으로 사용됩니다. geos, 또는  Kinhungsin은 각각 다른 이름으로 사용되기도 합니다.  ninglawaid에서 각각 다른 이름으로 사용될 수 있습니다. geos, 또는  Ninglawaid에서는 주로 불고기용으로 사용되는 경우가 많습니다. geos, 또는 geos는 각각 다른 이름으로 사용될 수 있습니다. geos, 또는 geos는 각각 다른 이름으로 쓰일 수 있습니다. geos, 또는 geos는 각각 다른 이름으로 사용되기도 합니다. neos, 또는 

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):"저는 데이터에 대한 의존도를 가지고 있지 않기 때문에 제임스 닉슨의 부통령직에서 근무한 기간도에 대해 알지 못합니다. 정확한 정보와 관련해서는 "리처드 닉슨의 47대 부통령직"이라는 개인의 이름이나 연도, 인물 정보에 대한 정보가 없습니다.", 'token': 147}自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由

### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어

### Response(응답):'저는 인공지능 챗봇이므로 시카고에 있는 국제공항 정보나 위치 정보가 없습니다. 하지만 현재 시카고에는 수많은 공항이 있으므로 

# 회고
- RLHF를 더 자세히 알 수 있었음
    - 처음에는 모델의 모든 대답을 사람이 평가하는 것으로 생각함
    - 하지만 일부 데이터셋에서만 라벨링을 한 후에 Reward Model을 만드는 방식이 신박하게 느껴짐
- LLM에 와서는 이제 학습 환경에 대해 신경을 써야함
    - 이제는 하나의 모델을 Fine-Tuning하는 것 조차 여러개의 모델을 필요로 함
    - GPU 환경에서 어떻게 학습할지 결정하기 위해서는 모델 학습이나 추론은 GPU에게 어떻게 맡기는지에 대한 지식이 필요함